In [1]:
import pyspark
from pyspark.sql import SparkSession
import findspark
findspark.init()

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("INFO")

In [3]:
# !wget !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [4]:
!wc -l fhvhv_tripdata_2021-01.csv.gz

508066 fhvhv_tripdata_2021-01.csv.gz


In [5]:
df = spark.read \
     .option("header", "true") \
     .csv("fhvhv_tripdata_2021-01.csv.gz")

In [6]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [7]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [8]:
import gzip, shutil

with gzip.open('fhvhv_tripdata_2021-01.csv.gz', 'r') as f_in, open('fhvhv_tripdata_2021-01.csv', 'wb') as f_out:
  shutil.copyfileobj(f_in, f_out)

In [9]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
import pandas as pd

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas = df_pandas.drop(columns=['SR_Flag'])

In [13]:
df_pandas.dtypes

hvfhs_license_num       object
dispatching_base_num    object
pickup_datetime         object
dropoff_datetime        object
PULocationID             int64
DOLocationID             int64
dtype: object

In [16]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True)])

In [18]:
from pyspark.sql import types

In [19]:
schema = types.StructType([types.StructField('hvfhs_license_num', types.StringType(), True), 
         types.StructField('dispatching_base_num', types.StringType(), True), 
         types.StructField('pickup_datetime', types.TimestampType(), True), 
         types.StructField('dropoff_datetime', types.TimestampType(), True), 
         types.StructField('PULocationID', types.IntegerType(), True), 
         types.StructField('DOLocationID', types.IntegerType(), True)
])

In [20]:
df = spark.read \
     .option("header", "true") \
     .schema(schema) \
     .csv('fhvhv_tripdata_2021-01.csv')

In [21]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-----------------+--------------------+-------------------+-------------------+------------+------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|          88|          42|
|           HV0005|              B025

In [23]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 48, 14), dropoff_

In [24]:
df = df.repartition(24)

In [25]:
df.write.parquet('fhvh/2021/01')

In [26]:
df = spark.read.parquet('fhvh/2021/01/')

In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)



In [29]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num=="HV0003") \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 04:13:30|2021-01-01 04:21:25|         235|         247|
|2021-01-06 21:50:40|2021-01-06 21:58:34|         142|         151|
|2021-01-02 13:38:20|2021-01-02 14:01:12|          50|         193|
|2021-01-06 16:17:49|2021-01-06 16:29:41|         234|         144|
|2021-01-01 17:32:22|2021-01-01 17:44:52|          35|          76|
|2021-01-06 16:14:50|2021-01-06 16:35:35|         233|         144|
|2021-01-01 17:48:35|2021-01-01 18:05:42|         148|         164|
|2021-01-02 15:52:27|2021-01-02 16:09:17|         107|         161|
|2021-01-02 16:38:16|2021-01-02 17:06:04|          41|         159|
|2021-01-05 14:44:47|2021-01-05 14:54:07|          61|          61|
|2021-01-02 00:00:25|2021-01-02 00:05:46|          78|          78|
|2021-01-01 18:21:29|2021-01-01 18:44:55|       

In [30]:
from pyspark.sql import functions

In [32]:
df \
    .withColumn('pickup_date', functions.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', functions.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|         235|         247|
| 2021-01-01|  2021-01-01|         247|         243|
| 2021-01-06|  2021-01-06|         142|         151|
| 2021-01-05|  2021-01-05|         100|         265|
| 2021-01-02|  2021-01-02|          50|         193|
| 2021-01-03|  2021-01-03|         234|         234|
| 2021-01-03|  2021-01-03|         168|          47|
| 2021-01-06|  2021-01-06|         234|         144|
| 2021-01-01|  2021-01-01|          35|          76|
| 2021-01-06|  2021-01-06|         233|         144|
| 2021-01-05|  2021-01-05|         235|         220|
| 2021-01-01|  2021-01-01|         148|         164|
| 2021-01-02|  2021-01-02|         107|         161|
| 2021-01-02|  2021-01-02|          41|         159|
| 2021-01-05|  2021-01-05|          61|          61|
| 2021-01-02|  2021-01-02|          78|       